In [92]:
from typing import Tuple
from dataclasses_json import dataclass_json
import json

import dataclasses
from pydantic import BaseModel
from pydantic import dataclasses as pddataclasses

In [93]:
class PydanticSphere(BaseModel):
    center: Tuple[float, float, float]
    radius: float

@pddataclasses.dataclass
class PydanticDCSphere:
    center: Tuple[float, float, float]
    radius: float

@dataclasses.dataclass
class DataclassSphere:
    center: Tuple[float, float, float]
    radius: float

models = (PydanticSphere, DataclassSphere, PydanticDCSphere)

In [94]:
# winner: dataclasses, pydantic.dataclasses
print('positional arguments')
for m in models:
    print('---')
    try:
        sphere = m((1,1,1), 1.0)
        print(sphere)
        print(f'\tworked for {m.__name__}')
    except Exception as e:
        print(f'\terrored for {m.__name__}')
        print(f'\terror = {e}')

positional arguments
---
	errored for PydanticSphere
	error = __init__() takes exactly 1 positional argument (3 given)
---
DataclassSphere(center=(1, 1, 1), radius=1.0)
	worked for DataclassSphere
---
PydanticDCSphere(center=(1.0, 1.0, 1.0), radius=1.0)
	worked for PydanticDCSphere


In [95]:
# winner: pydantic, pydantic.dataclasses
print('type conversion')
for m in models:
    print('---')
    try:
        sphere = m(center=(1,1,1), radius=1)
        print(sphere)
        assert isinstance(sphere.radius, float),  "radius not converted to float"
        print(f'\tworked for {m.__name__}')
    except Exception as e:
        print(f'\terrored for {m.__name__}')
        print(f'\terror = {e}')

type conversion
---
center=(1.0, 1.0, 1.0) radius=1.0
	worked for PydanticSphere
---
DataclassSphere(center=(1, 1, 1), radius=1)
	errored for DataclassSphere
	error = radius not converted to float
---
PydanticDCSphere(center=(1.0, 1.0, 1.0), radius=1.0)
	worked for PydanticDCSphere


In [96]:
# winner: pydantic, pydantic.dataclasses
print('catches wrong types conversion')
for m in models:
    print('---')
    try:
        sphere = m(center=('s',1,1), radius=1.0)
        print(sphere)
        print(f'\tworked for {m.__name__}')
    except Exception as e:
        print(f'\terrored for {m.__name__}')
        print(f'\terror = {e}')

catches wrong types conversion
---
	errored for PydanticSphere
	error = 1 validation error for PydanticSphere
center -> 0
  value is not a valid float (type=type_error.float)
---
DataclassSphere(center=('s', 1, 1), radius=1.0)
	worked for DataclassSphere
---
	errored for PydanticDCSphere
	error = 1 validation error for PydanticDCSphere
center -> 0
  value is not a valid float (type=type_error.float)
